## Import the Libraries

In [4]:
import pandas as pd
import numpy as np
import os
import requests
import zipfile
import io
import re

## Extracting Sample files for the year in range 1999-2016

In [2]:
def validate_password(value):
    while True:
        try:
            password = (input(value))
            if len(password) == 0:
                    print("Sorry, Incorrect password!, Please provide a valid password.")
                    continue
                    #better try again... Return to the start of the loop
        except ValueError:
            print("Sorry, Incorrect password! Please provide a valid password")
                #better try again... Return to the start of the loop
            continue
        else:
                #we're ready to exit the loop.
            break
    return password

In [5]:
pattern = r"\"?([-a-zA-Z0-9.`?{}]+@\w+\.\w+)\"?"
EMAIL_REGEX = re.compile(pattern)

def validate_email(value):
    while True:
        try:
            email = (input(value))
            if not EMAIL_REGEX.match(email):
                print("Sorry, Incorrect add! Please provide a valid add") 
                continue
                #if len(password) == 0:
                    #print("Sorry, Incorrect password!, Please provide a valid password.")
                    #continue
                    #better try again... Return to the start of the loop
        except ValueError:
            print("Sorry, Incorrect add! Please provide a valid add")
                #better try again... Return to the start of the loop
            continue
        else:
                #adderss was successfully parsed!
                #we're ready to exit the loop.
            break
    return email

In [ ]:
def validate_year(value):
    while True:
        try:
            email = (input(value))
            if not EMAIL_REGEX.match(email):
                print("Sorry, Incorrect add! Please provide a valid add") 
                continue
                #if len(password) == 0:
                    #print("Sorry, Incorrect password!, Please provide a valid password.")
                    #continue
                    #better try again... Return to the start of the loop
        except ValueError:
            print("Sorry, Incorrect add! Please provide a valid add")
                #better try again... Return to the start of the loop
            continue
        else:
                #adderss was successfully parsed!
                #we're ready to exit the loop.
            break
    return email

In [6]:
emailadd = validate_email('Enter address:')
paswd = validate_password('Enter password:')

Enter address:jain.yo@husky.neu.edu
Enter password:0_iSRa}e


In [12]:
def user_input_validation(value):
    while True:
        try:
            curr_yr = int(input(value))
            if curr_yr not in range(1999,2017):
                print("Sorry, Incorrect Year! Please provide a valid year") 
                continue
                #better try again... Return to the start of the loop
        except ValueError:
            print("Sorry, Incorrect Year! Please provide a valid year")
            #better try again... Return to the start of the loop
            continue
        else:
            #year was successfully parsed!
            #we're ready to exit the loop.
            break
    return curr_yr

year = user_input_validation('Input Year:')


Input Year:123
Sorry, Incorrect Year! Please provide a valid year
Input Year:2003


In [8]:
s = requests.session()
payload = {'username': emailadd, 'password':paswd}
url = 'https://freddiemac.embs.com/FLoan/secure/auth.php'
a = s.post(url, data=payload)

values = {'accept': 'Yes','acceptSubmit':'Continue','action': 'acceptTandC'}
url1 = 'https://freddiemac.embs.com/FLoan/Data/download.php'
b = s.post(url1,data=values)


In [ ]:
if str(year) == '2016':
    url3 = 'https://freddiemac.embs.com/FLoan/Data/historical_data1_Q1'+''+str(year)+'.zip'
    file = s.get(url3)
    m=zipfile.ZipFile(io.BytesIO(file.content))
    m.extractall()
else:
    for quarter in range(1,5):
        url3 = 'https://freddiemac.embs.com/FLoan/Data/historical_data1_Q'+str(quarter)+''+str(year)+'.zip'
        print('downloading files..,Please be patient..!')
        file = s.get(url3)
        m=zipfile.ZipFile(io.BytesIO(file.content))
        m.extractall()

# Get all the original sample files

In [28]:
original_files =[file for file in os.listdir() if file.startswith('historical_data1_Q') ]
original_files

['historical_data1_Q12005.txt']

# Reading files one by one

In [120]:
def load_df(file):
    df = pd.read_table(file,delimiter='|', header=None )
    return df

In [121]:
def name_columns(df):
    df.columns = ['CREDIT_SCORE','FIRST_PAYMENT_DATE','FIRST_TIME_HOMEBUYER_FLAG','MATURITY_DATE','MSA','MORTGAGE_INSURANCE_PERCENTAGE_MI','NUMBER_OF_UNITS','OCCUPANCY_STATUS','ORIGINAL_COMBINED_LOAN_TO_VALUE_CLTV','ORIGINAL_DEBT_TO_INCOME_DTI_RATIO','ORIGINAL_UPB','ORIGINAL_LOAN_TO_VALUE_LTV','ORIGINAL_INTEREST_RATE','CHANNEL','PREPAYMENT_PENALTY_MORTGAGE_PPM_FLAG','PRODUCT_TYPE','PROPERTY_STATE','PROPERTY_TYPE','POSTAL_CODE','LOAN_SEQUENCE_NUMBER','LOAN_PURPOSE','ORIGINAL_LOAN_TERM','NUMBER_OF_BORROWERS','SELLER_NAME','SERVICER_NAME','SUPER_CONFORMING_FLAG']
    numberofrows = len(df.index)
    print (numberofrows)
    return df

In [122]:
def validate_creditScore(df):
    credit_score_count = df.groupby('CREDIT_SCORE')
    #Check if there is value less than 300
    print (credit_score_count['CREDIT_SCORE'].count().head(n=5))
    #Check if there is value greater than 850
    print (credit_score_count['CREDIT_SCORE'].count().tail(n=5))
    
    #Replace Blanks with 0 for Credit Score Column and check condition for < 300 and > 850¶
    
    df['CREDIT_SCORE']  = df['CREDIT_SCORE'].convert_objects(convert_numeric=True).fillna(0)
    df['CREDIT_SCORE'] = df['CREDIT_SCORE'].astype(int)
    df.CREDIT_SCORE[df.CREDIT_SCORE < 300] = 0
    df.CREDIT_SCORE[df.CREDIT_SCORE > 850] = 0
    credit_score_count1 = df.groupby('CREDIT_SCORE')
    #Check if there is value less than 300
    print (credit_score_count1['CREDIT_SCORE'].count().head(n=5))
    #Check if there is value greater than 850
    print (credit_score_count1['CREDIT_SCORE'].count().tail(n=5))
    
    return df

In [123]:
def validate_first_payment_date(df):
    df['FIRST_PAYMENT_DATE'].isnull().values.any()
    return df

### 3) FIRST TIME HOMEBUYER FLAG

Indicates whether the Borrower, or one of a group of Borrowers, is an individual who (1) is purchasing the mortgaged property, (2) will reside in the mortgaged property as a primary residence and (3) had no ownership interest (sole or joint) in a residential property during the three-year period preceding the date of the purchase of the mortgaged property. With certain limited exceptions, a displaced homemaker or single parent may also be considered a First-Time Homebuyer if the individual had no ownership interest in a residential property during the preceding three-year period other than an ownership interest in the marital residence with a spouse.  
 
Investment Properties, Second Homes and Refinance transactions are not eligible to be considered First-Time Homebuyer transactions. 

Therefore First Time Homebuyer does not apply and will be disclosed as “Not Applicable”, which will be indicated by a blank space. 

In [124]:
def validate_first_time_homebuyer_flag(df):
    #Check for unique values
    #df.FIRST_TIME_HOMEBUYER_FLAG.unique()
    #Fill nan as Unknown
    df['FIRST_TIME_HOMEBUYER_FLAG'] = df.FIRST_TIME_HOMEBUYER_FLAG.fillna('U')
    #first_time_home_buyer_flag_count = df.groupby('FIRST_TIME_HOMEBUYER_FLAG')
    #print (first_time_home_buyer_flag_count['FIRST_TIME_HOMEBUYER_FLAG'].count())
    
    return df

### 4) MATURITY DATE

The month in which the final monthly payment on the mortgage is scheduled to be made as stated on the original mortgage note

In [125]:
def validate_maturity_date(df):
    if df['MATURITY_DATE'].isnull().values.any():
        df['MATURITY_DATE']= df.MATURITY_DATE.fillna('199701')
    
    return df

### 5) METROPOLITAN STATISTICAL AREA (MSA) OR METROPOLITAN DIVISION

This disclosure will be based on the designation of the Metropolitan Statistical Area or Metropolitan Division based on 2010 census (for Mar 2013 and May 2013 releases) and 2013 census (for Aug 2013 and Dec 2013 releases) data. Metropolitan Statistical Areas (MSAs) are defined by the United States Office of Management and Budget (OMB) and have at least one urbanized area with a population of 50,000 or more inhabitants. OMB refers to an MSA containing a single core with a population of 2.5 million or more, which may be comprised of groupings of counties, as a Metropolitan Division. 
 
If an MSA applies to a mortgaged property, the applicable five-digit value is disclosed; however, if the mortgaged property also falls within a Metropolitan Division classification, the applicable five-digit value for the Metropolitan Division takes precedence and is disclosed instead. 
 
Changes and/or updates in designations of MSAs or Metropolitan Division will not be reflected in the Single Family Historical Dataset.

1) Metropolitan Division or MSA Code.  
2) 00000 = Indicates that the area in which the mortgaged property is located is
   a) neither an MSA nor a Metropolitan Division, or b) unknown. 

In [126]:
def validate_msa(df):
    try:
        if any(df['MSA'].isnull()):
            df['MSA'] = df.MSA.fillna(00000)
    except:
        pass
    return df

### 6) MORTGAGE INSURANCE PERCENTAGE (MI %)

The percentage of loss coverage on the loan, at the time of Freddie Mac’s purchase of the mortgage loan that a mortgage insurer is providing to cover losses incurred as a result of a default on the loan.  Only primary mortgage insurance that is purchased by the Borrower, lender or Freddie Mac is disclosed. Mortgage insurance that constitutes “credit enhancement” that is not required by Freddie Mac’s Charter is not disclosed.  
 
Amounts of mortgage insurance reported by Sellers that are less than 1% or greater than 55% will be disclosed as “Unknown,” which will be indicated by three blank spaces.  No MI will be indicated by three zeros

1% - 55%  000 =  No MI  111 = Unknown

Is 0 same as 000 ?

In [127]:
def validate_mortgage(df):
    df['MORTGAGE_INSURANCE_PERCENTAGE_MI'] = df.MORTGAGE_INSURANCE_PERCENTAGE_MI.replace('   ',111)    
    if any(df['MORTGAGE_INSURANCE_PERCENTAGE_MI'].isnull()):
            df['MORTGAGE_INSURANCE_PERCENTAGE_MI']  = df['MORTGAGE_INSURANCE_PERCENTAGE_MI'].convert_objects(convert_numeric=True).fillna(111)
            
            if df.MORTGAGE_INSURANCE_PERCENTAGE_MI.values.any()>55:
                df['MORTGAGE_INSURANCE_PERCENTAGE_MI']=111
                df['MORTGAGE_INSURANCE_PERCENTAGE_MI'] = df['MORTGAGE_INSURANCE_PERCENTAGE_MI'].astype(int)        
            
    print('returning df')
    return df

### 7) NUMBER OF UNITS

Denotes whether the mortgage is a one-, two-, three-, or four-unit property.
three-, or four-unit property. 
 
1 = one-unit  2 = two-unit  3 = three-unit  4 = four-unit  0 = Unknown

In [128]:
def validate_number_of_units(df):
    #Replace nan as 0 which is Unknown
    if any(df['NUMBER_OF_UNITS'].isnull()):
        df['NUMBER_OF_UNITS'] = df.NUMBER_OF_UNITS.fillna(0)
    
    return df

### 8) OCCUPANCY STATUS

Denotes whether the mortgage type is owner occupied, second home, or investment property.  

O = Owner Occupied  I = Investment Property  S = Second Home  U  = Unknown

In [129]:
def validate_occupany_status(df):
    if any(df['OCCUPANCY_STATUS'].isnull()):
        df['OCCUPANCY_STATUS'] = df.OCCUPANCY_STATUS.fillna(0)
    return df
    

### 9) ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)

In the case of a purchase mortgage loan, the ratio is obtained by dividing the original mortgage loan amount on the note date plus any secondary mortgage loan amount disclosed by the Seller by the lesser of the mortgaged property’s appraised value on the note date or its purchase price.  In the case of a refinance mortgage loan, the ratio is obtained by dividing the original mortgage loan amount on the note date plus any secondary mortgage loan amount disclosed by the Seller by the mortgaged property’s appraised value on the note date.  If the secondary financing amount disclosed by the Seller includes a home equity line of credit, then the CLTV calculation reflects the disbursed amount at closing of the first lien mortgage loan, not the maximum loan amount available under the home equity line of credit.  In the case of a seasoned mortgage loan, if the Seller cannot warrant that the value of the mortgaged property has not declined since the note date, Freddie Mac requires that the Seller must provide a new appraisal value, which is used in the CLTV calculation.  In certain cases, where the Seller delivered a loan to Freddie Mac with a special code indicating additional secondary mortgage loan amounts, those amounts may have been included in the CLTV calculation.    
 
If the calculated CLTV is < 0 or > 200, and the LTV is > 80 and <= 200 set the CLTV to the LTV.  If the LTV is < 80 or > 200 or unknown, set the CLTV to ‘unknown.’  If the CLTV is < LTV, set the CLTV to ‘unknown.’  Unknown is indicated by a null value. 
 
This disclosure is subject to the widely varying standards originators use to verify Borrowers’ secondary mortgage loan amounts and will not be updated. 

0% - 200%  Null = Unknown = 999

In [130]:
def validate_cltv(df):
    if any(df['ORIGINAL_COMBINED_LOAN_TO_VALUE_CLTV'].isnull()):
        df['ORIGINAL_COMBINED_LOAN_TO_VALUE_CLTV'] = df.ORIGINAL_COMBINED_LOAN_TO_VALUE_CLTV.fillna(999)
    
    return df
    
    

### 10) ORIGINAL DEBT-TO-INCOME (DTI) RATIO 

Disclosure of the debt to income ratio is based on (1) the sum of the borrower's monthly debt payments, including monthly housing expenses that incorporate the mortgage payment the borrower is making at the time of the delivery of the mortgage loan to Freddie Mac, divided by (2) the total monthly income used to underwrite the borrower as of the date of the origination of the mortgage loan.  
 
Ratios greater than 65% are indicated by three (3) blank spaces and if unknown are indicated by a Null value. 
 
This disclosure is subject to the widely varying standards originators use to verify Borrowers’ assets and liabilities and will not be updated

0%< DTI <= 65%  Space(3) >65% = 70 Null = Unknown = 99

In [131]:
def validate_dti(df):
    if any(df['ORIGINAL_DEBT_TO_INCOME_DTI_RATIO'].isnull()):
        df['ORIGINAL_DEBT_TO_INCOME_DTI_RATIO'] = df.ORIGINAL_DEBT_TO_INCOME_DTI_RATIO.fillna(99)
        
    df['ORIGINAL_DEBT_TO_INCOME_DTI_RATIO'] = df.ORIGINAL_DEBT_TO_INCOME_DTI_RATIO.replace('   ',70)
        #ORIGINAL_DEBT_TO_INCOME_DTI_RATIO_count = df.groupby('ORIGINAL_DEBT_TO_INCOME_DTI_RATIO')
        #print (ORIGINAL_DEBT_TO_INCOME_DTI_RATIO_count['ORIGINAL_DEBT_TO_INCOME_DTI_RATIO'].count().head())
        
    return df
    

### 12) ORIGINAL LOAN-TO-VALUE (LTV) 

In the case of a purchase mortgage loan, the ratio obtained by dividing the original mortgage loan amount on the note date by the lesser of the mortgaged property’s appraised value on the note date or its purchase price. 
 
In the case of a refinance mortgage loan, the ratio obtained by dividing the original mortgage loan amount on the note date and the mortgaged property’s appraised value on the note date. 
 
In the case of a seasoned mortgage loan, if the Seller cannot warrant that the value of the mortgaged property has not declined since the note date, Freddie Mac requires that the Seller must provide a new appraisal value, which is used in the LTV calculation. 
 
Ratios below 6% or greater than 105% will be disclosed as “Unknown,” indicated by three (3) blank spaces = 999.

In [132]:
def validate_ltv(df):
    if any(df['ORIGINAL_LOAN_TO_VALUE_LTV'].isnull()):
        
        df['ORIGINAL_LOAN_TO_VALUE_LTV'] = df.ORIGINAL_LOAN_TO_VALUE_LTV.fillna(999)
        df['ORIGINAL_LOAN_TO_VALUE_LTV'] = df.ORIGINAL_LOAN_TO_VALUE_LTV.astype(int)
        
    #if df.ORIGINAL_LOAN_TO_VALUE_LTV.values.any() < 6 or df.ORIGINAL_LOAN_TO_VALUE_LTV.values.any() < 105:
        #df['ORIGINAL_LOAN_TO_VALUE_LTV']= 999
            
        
    return df       

### 13) ORIGINAL INTEREST RATE 

The original note rate as indicated on the mortgage note. 

In [133]:
def valdiate_interest_rate(df):
    if any(df['ORIGINAL_INTEREST_RATE'].isnull()):
        df['ORIGINAL_INTEREST_RATE'].fillna()
        
    return df   

### 14) CHANNEL 

R = Retail  B = Broker  C = Correspondent  T = TPO Not Specified  Space = Unknown = U

In [134]:
def validate_channel(df):
    if any(df['CHANNEL'].isnull()):
        df['CHANNEL'] = df.CHANNEL.fillna('U')
        
    df['CHANNEL'] = df.CHANNEL.replace(' ','U')
        
    return df
    

### 15) PREPAYMENT PENALTY MORTGAGE (PPM) FLAG 

Denotes whether the mortgage is a PPM. A PPM is a mortgage with respect to which the borrower is, or at any time has been, obligated to pay a penalty in the event of certain repayments of principal. 

Y = PPM  N = Not PPM   Space (1) = Unknown = U

In [135]:
def validate_ppm(df):
    if any(df['PREPAYMENT_PENALTY_MORTGAGE_PPM_FLAG'].isnull()):
        df['PREPAYMENT_PENALTY_MORTGAGE_PPM_FLAG'] = df.PREPAYMENT_PENALTY_MORTGAGE_PPM_FLAG.fillna('U')
    return df
        
        
    

### 16) PRODUCT TYPE 

Denotes that the product is a fixed-rate mortgage.

FRM – Fixed Rate Mortgage 

In [136]:
def validate_productType(df):
    return df


### 17) PROPERTY STATE 

A two-letter abbreviation indicating the state or territory within which the property securing the mortgage is located. 

AL, TX, VA, etc
UN = Unknown

In [137]:
def validate_property_state(df):
    if any(df['PROPERTY_STATE'].isnull()):
        df['PROPERTY_STATE'] = df.PROPERTY_STATE.fillna('UN')
    return df
        
    

### 18) PROPERTY TYPE 

Denotes whether the property type secured by the mortgage is a condominium, leasehold, planned unit development (PUD), cooperative share, manufactured home, or Single Family home.  
 
If the Property Type is unknown, this will be indicated by two (2) blank spaces or 'UN'.

CO = Condo  LH = Leasehold  PU = PUD  MH = Manufactured Housing  SF = 1-4 Fee Simple  CP = Co-op  
Space (2) = Unknown = UN

In [138]:
def validate_property_type(df):
    if any(df['PROPERTY_TYPE'].isnull()):
        df['PROPERTY_TYPE'] = df.PROPERTY_TYPE.replace('  ','UN')
        
    return df

### 19) POSTAL CODE 

The postal code for the location of the mortgaged property 

###00, where “###” represents the first three digits of the 5digit postal code  
Space(5)= Unknown = 99999

In [139]:
def validate_postal_code(df):
    if any(df['POSTAL_CODE'].isnull()):
        df['POSTAL_CODE'] = df.POSTAL_CODE.fillna(99999)
    return df
        

### 20) LOAN SEQUENCE NUMBER 

Unique identifier assigned to each loan

F1YYQnXXXXXX 
 
F1 = product (Fixed Rate Mortgage);   YYQn = origination year and quarter; and,   XXXXXX = randomly assigned digits

In [140]:
def validate_loan_seq_number(df):
    return df
    ######### validation needed..!!!
    

### 21) LOAN PURPOSE

P = Purchase  C = Cash-out Refinance  N = No Cash-out Refinance  Space = Unknown = U

In [141]:
def validate_loan_purpose(df):
    if any(df['LOAN_PURPOSE'].isnull()):
        df['LOAN_PURPOSE'] = df.LOAN_PURPOSE.fillna('U')
    return df
    

### 22) ORIGINAL LOAN TERM

A calculation of the number of scheduled monthly payments of the mortgage based on the First Payment Date and Maturity Date. 
 
Loans with original term of 420 or more, or 300 or less, are excluded from the Dataset if originated prior to 1/1/2005.  If loan was originated on/after 1/1/2005, this exclusion does not apply.

Calculation:  (Loan Maturity Date (MM/YY) – Loan First Payment Date (MM/YY) + 1) 

In [142]:
def validate_original_loan_term(df):
    return df
    
        ### validation required!!
    

### 23) Number of Borrowers

The number of Borrower(s) who are obligated to repay the mortgage note secured by the mortgaged property. Disclosure denotes only whether there is one borrower or more than one borrower associated with the mortgage note. This disclosure will not be updated to reflect any subsequent assumption of the mortgage note. 

01 = 1 borrower  02 = > 1 borrowers  Space (2) = Unknown = 'UN'

In [143]:
def validate_number_of_borrowers(df):
    if any(df['NUMBER_OF_BORROWERS'].isnull()):
            df['NUMBER_OF_BORROWERS'] = df.NUMBER_OF_BORROWERS.fillna('UN')
            #print (sum(number_of_borrowers_count['NUMBER_OF_BORROWERS'].count()))
    return df

### 24) SELLER NAME

The entity acting in its capacity as a seller of mortgages to Freddie Mac at the time of acquisition Seller Name will be disclosed for sellers with a total Original UPB representing 1% or more of the total Original UPB of all loans in the Dataset for a given calendar quarter.  Otherwise, the Seller Name will be set to “Other Sellers”. 

Name of the seller, or “Other Sellers”

In [144]:
def validate_seller_name(df):
    if any(df['SELLER_NAME'].isnull()):
        df['SELLER_NAME'] = df.SELLER_NAME.fillna('Other sellers')
        #print (sum(df['SELLER_NAME'].count()))
        
    return df
        

### 25) SERVICER NAME 

The entity acting in its capacity as the servicer of mortgages to Freddie Mac as of the last period for which loan activity is reported in the Dataset.   
 
Servicer Name will be disclosed for servicers with a total Original UPB representing 1% or more of the total Original UPB of all loans in the Dataset for a given calendar quarter.  Otherwise, the Servicer Name will be set to “Other Servicers”. 

Name of the servicer, or “Other Servicers” 

In [145]:
def validate_service_name(df):
    if any(df['SERVICER_NAME'].isnull()):
        df['SERVICER_NAME'] = df.SERVICER_NAME.fillna('Other servicers')
        #print (sum(servicer_name_count['SERVICER_NAME'].count()))
        
    return df

### 26) Super Conforming Flag

For mortgages that exceed conforming loan limits with origination dates on or after 10/1/2008 and settlements on or after 1/1/2009 

Y = Yes  Space (1)  = Not Super Conforming = N

In [146]:
def validate_super_confirmng_flag(df):
    if any(df['SUPER_CONFORMING_FLAG'].isnull()):
        df['SUPER_CONFORMING_FLAG'] = df.SUPER_CONFORMING_FLAG.fillna('N')
        #print (sum(scf_count['SUPER_CONFORMING_FLAG'].count()))
        
    return df

In [ ]:
for i in range(len(original_files)):
    df = pd.DataFrame()
    print (original_files[i])
    df = load_df(original_files[i])
    
    #27)
    df = name_columns(df)
    
     # 2)
    df = validate_creditScore(df)
    
    # 3) (check)
    df = validate_dti(df)
    
    #4)  (check)
    df = validate_ltv(df)
    
    #5) (check)
    df = valdiate_interest_rate(df)
    
    #6) (check)
    df = validate_channel(df)
    
    #7) (check)
    df = validate_ppm(df)
    
    #8) (check)
    df = validate_property_state(df)
    
    #9) (check)
    df = validate_property_type(df)
    
    #10) (check)
    df = validate_first_payment_date(df)
    
    #11) (check)
    df = validate_postal_code(df) 
    
    #12) (check)
    df = validate_loan_purpose(df)
    
    #13) (check)
    df = validate_number_of_borrowers(df)
    
    #14) (check)
    df = validate_seller_name(df)
    
    #15) (check)
    df = validate_service_name(df)
     
    #16) 
    df = validate_super_confirmng_flag(df)
    
    #17) (check)
    df = validate_first_time_homebuyer_flag(df)
    
    #18) (check)
    df = validate_maturity_date(df)
    
    #19) (check)
    df = validate_msa(df)
    
    #20) (check)
    df = validate_number_of_units(df)
    
    #21) (check)
    df = validate_occupany_status(df)
    
    #22) (check)
    df = validate_cltv(df)
    
    #23) (check)
    df = validate_productType(df)
    
    #24) (check)
    df = validate_loan_seq_number(df)
    
    #25) (check)
    df = validate_original_loan_term(df)
  
 
    df.to_csv('Sample_Original_Validated_'+str(original_files[i][12:-4])+'.csv', index=None)

In [115]:
print ("All Files have been Validated !!")

All Files have been Validated !!


In [ ]:
for i in range(len(original_files)):
    df = pd.DataFrame()
    print (original_files[i])
    df = load_df(original_files[i])
    df = name_columns(df)
    df = validate_creditScore(df)
    df = validate_dti(df)
    df = validate_ltv(df)
    df = valdiate_interest_rate(df)
    df = validate_channel(df)
    df = validate_ppm(df)
    df = validate_property_state(df)
    df = validate_property_type(df)
    df = validate_first_payment_date(df)
    df = validate_postal_code(df) 
    df = validate_loan_purpose(df)
    df = validate_number_of_borrowers(df)
    df = validate_seller_name(df)
    df = validate_service_name(df)
    df = validate_super_confirmng_flag(df)
    df = validate_first_time_homebuyer_flag(df)
    
    #18) (check)
    df = validate_maturity_date(df)
    
    #19) (check)
    df = validate_msa(df)
    
    #20) (check)
    df = validate_number_of_units(df)
    
    #21) (check)
    df = validate_occupany_status(df)
    
    #22) (check)
    df = validate_cltv(df)
    
    #23) (check)
    df = validate_productType(df)
    
    #24) (check)
    df = validate_loan_seq_number(df)
    
    #25) (check)
    df = validate_original_loan_term(df)
  
 
    df.to_csv('Sample_Original_Validated_'+str(original_files[i][12:-4])+'.csv', index=None)